In [1]:
# ============================================================
#  DSA210 Term Project
#  Comparing National Olympic Success and GDP
#  Student: Alp Orkun Güzel - 34524
# ============================================================

# ------------------------------------------------------------
# AŞAMA 0: Gerekli kütüphaneler
# ------------------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr, ttest_ind
import statsmodels.api as sm

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 200)

# Eğer seaborn/statsmodels yüklü değilse (Colab hariç):
# !pip install seaborn statsmodels


# ------------------------------------------------------------
# AŞAMA 1: Veri Yükleme
# ------------------------------------------------------------
# 1.1 Olympic athlete_events dataset (Kaggle)
# Bu dosyanın adını kendi bilgisayarındaki isimle değiştir:
olympics_path = "athlete_events.csv"      # <-- gerekirse değiştir
noc_regions_path = "noc_regions.csv"      # Kaggle Olympic dataset ile birlikte gelir

olympics = pd.read_csv(olympics_path)
noc_regions = pd.read_csv(noc_regions_path)

print("Olympics shape:", olympics.shape)
print("NOC regions shape:", noc_regions.shape)
display(olympics.head())
display(noc_regions.head())


# ------------------------------------------------------------
# AŞAMA 2: Veri Ön İşleme - Olympics
# ------------------------------------------------------------
# Sadece Summer Olympics alalım (istersen değiştirilebilir)
olympics = olympics[olympics["Season"] == "Summer"]

# NOC kodlarını ülke isimlerine map etmek için noc_regions kullanıyoruz
olympics = olympics.merge(noc_regions[["NOC", "region"]], on="NOC", how="left")
olympics.rename(columns={"region": "Country_Name"}, inplace=True)

# Madalyası olmayan satırları at (Medal NaN ise, madalya yok)
medals = olympics.dropna(subset=["Medal"])

# Yıl filtresi: Örneğin 1980 sonrası çalışmak istiyorsan:
min_year = 1980
medals = medals[medals["Year"] >= min_year]

print("Filtered medals shape:", medals.shape)
display(medals.head())


# ------------------------------------------------------------
# AŞAMA 3: Ülke-Yıl Bazında Madalya Sayımı
# ------------------------------------------------------------
medals_country_year = (
    medals
    .groupby(["NOC", "Country_Name", "Year"])["Medal"]
    .count()
    .reset_index()
    .rename(columns={"Medal": "Total_Medals"})
)

print("Medals by country-year shape:", medals_country_year.shape)
display(medals_country_year.head(10))


# ------------------------------------------------------------
# AŞAMA 4: World Bank Verilerini Yükleme
# ------------------------------------------------------------
# Burada World Bank'ten indirdiğin CSV dosyalarını kullanmanı öneriyorum.
# Genelde World Bank CSV formatı şöyle:
# Country Name | Country Code | Indicator Name | Indicator Code | 1960 | 1961 | ... | 2023
#
# Aşağıdaki fonksiyon, bu formatı uzun forma (Country, Year, Value) çeviriyor.

def load_wb_indicator(csv_path, value_name):
    """
    World Bank style wide CSV'yi (yıllar sütun) uzun formata çevirir.
    value_name: çıktı kolonu ismi (örn: 'GDP', 'GDP_per_capita', 'Population')
    """
    df = pd.read_csv(csv_path)
    
    # İlk 4 kolondan sonrasını yıl sütunları olarak al
    value_cols = df.columns[4:]
    
    df_long = df.melt(
        id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
        value_vars=value_cols,
        var_name="Year",
        value_name=value_name
    )
    
    # Yıl kolonunu integer yap
    df_long["Year"] = pd.to_numeric(df_long["Year"], errors="coerce")
    df_long = df_long.dropna(subset=["Year"])
    df_long["Year"] = df_long["Year"].astype(int)
    
    return df_long[["Country Name", "Country Code", "Year", value_name]]


# Bu csv dosyalarının isimlerini kendi indirdiğin dosyalara göre değiştir:
gdp_csv_path = "API_NY.GDP.MKTP.CD_DS2_en_csv_v2.csv"         # Toplam GDP
gdp_pc_csv_path = "API_NY.GDP.PCAP.CD_DS2_en_csv_v2.csv"      # GDP per capita
pop_csv_path = "API_SP.POP.TOTL_DS2_en_csv_v2.csv"            # Population

gdp_df = load_wb_indicator(gdp_csv_path, "GDP")
gdp_pc_df = load_wb_indicator(gdp_pc_csv_path, "GDP_per_capita")
pop_df = load_wb_indicator(pop_csv_path, "Population")

print("GDP shape:", gdp_df.shape)
print("GDP per capita shape:", gdp_pc_df.shape)
print("Population shape:", pop_df.shape)

display(gdp_df.head())
display(gdp_pc_df.head())
display(pop_df.head())


# ------------------------------------------------------------
# AŞAMA 5: World Bank Verilerini Birleştirme
# ------------------------------------------------------------
# GDP, GDP_per_capita ve Population'ı tek tabloya alalım
econ = gdp_df.merge(gdp_pc_df, on=["Country Name", "Country Code", "Year"], how="outer")
econ = econ.merge(pop_df, on=["Country Name", "Country Code", "Year"], how="outer")

# Yıl filtresi (Olympic verinle aynı aralıkta kalmak için):
econ = econ[econ["Year"] >= min_year]

print("Merged econ shape:", econ.shape)
display(econ.head(10))


# ------------------------------------------------------------
# AŞAMA 6: NOC -> Country Name eşleştirme ve merge
# ------------------------------------------------------------
# Olympic verisinde Country_Name, noc_regions'den geliyor (örneğin "United States", "Germany" gibi)
# World Bank'te "Country Name" var.
# Tam birebir eşleşmeyen bazı ülkeler olabilir, ama ana ülkeler için işe yarar.

# İlk olarak merge için basit bir eşleme deneyelim:
merged = medals_country_year.merge(
    econ,
    left_on=["Country_Name", "Year"],
    right_on=["Country Name", "Year"],
    how="left"
)

# Sadece isim kolonu karmaşasını azaltmak için:
merged = merged.drop(columns=["Country Name"])

print("Final merged shape:", merged.shape)
display(merged.head(20))

# Eksik değerleri kontrol edelim:
print("\nMissing values (first few columns):")
print(merged[["Country_Name", "Year", "Total_Medals", "GDP", "GDP_per_capita", "Population"]].isna().sum())


# ------------------------------------------------------------
# AŞAMA 7: Türev Değişkenler (Normalization)
# ------------------------------------------------------------
# Bazı ülkelerde GDP veya Population 0 veya NaN olabilir -> bölen yapmadan önce kontrol:
merged = merged.copy()

# Nüfus > 0 ise kişi başı madalya üretelim
merged["Medals_per_Million_People"] = np.where(
    (merged["Population"] > 0),
    merged["Total_Medals"] / (merged["Population"] / 1_000_000),
    np.nan
)

# GDP > 0 ise GDP başına madalya
merged["Medals_per_Billion_GDP"] = np.where(
    (merged["GDP"] > 0),
    merged["Total_Medals"] / (merged["GDP"] / 1_000_000_000),
    np.nan
)

display(merged.head(20))


# ------------------------------------------------------------
# AŞAMA 8: EDA - Genel Bakış
# ------------------------------------------------------------
print("Merged dataset info:")
print(merged.info())

print("\nSummary stats:")
display(merged[["Total_Medals", "GDP", "GDP_per_capita", "Population",
                "Medals_per_Million_People", "Medals_per_Billion_GDP"]].describe())


# ------------------------------------------------------------
# AŞAMA 9: EDA - Görselleştirme
# ------------------------------------------------------------

# 9.1 Medals vs GDP scatter
plt.figure()
plt.scatter(merged["GDP"], merged["Total_Medals"])
plt.xscale("log")   # GDP çok geniş, log scale daha iyi görünür
plt.xlabel("GDP (log scale)")
plt.ylabel("Total Medals")
plt.title("GDP vs Total Medals")
plt.tight_layout()
plt.show()

# 9.2 GDP per capita vs Medals
plt.figure()
plt.scatter(merged["GDP_per_capita"], merged["Total_Medals"])
plt.xscale("log")
plt.xlabel("GDP per capita (log scale)")
plt.ylabel("Total Medals")
plt.title("GDP per capita vs Total Medals")
plt.tight_layout()
plt.show()

# 9.3 Population vs Medals
plt.figure()
plt.scatter(merged["Population"], merged["Total_Medals"])
plt.xscale("log")
plt.xlabel("Population (log scale)")
plt.ylabel("Total Medals")
plt.title("Population vs Total Medals")
plt.tight_layout()
plt.show()

# 9.4 Korelasyon Heatmap
corr_cols = ["Total_Medals", "GDP", "GDP_per_capita", "Population",
             "Medals_per_Million_People", "Medals_per_Billion_GDP"]

plt.figure(figsize=(8, 6))
sns.heatmap(merged[corr_cols].corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix")
plt.tight_layout()
plt.show()


# ------------------------------------------------------------
# AŞAMA 10: Hipotez Testi 1
# H0: GDP ile medalya sayısı arasında ilişki yok (ρ = 0)
# H1: GDP ile medalya sayısı arasında pozitif korelasyon var (ρ > 0)
# ------------------------------------------------------------
# Korelasyon için verileri temizleyelim:
subset = merged[["GDP", "Total_Medals"]].dropna()
x = subset["GDP"]
y = subset["Total_Medals"]

corr, pval = pearsonr(np.log10(x), y)  # log(GDP) kullanmak genelde daha stabil
print("\nHipotez 1 - Pearson correlation (log GDP vs medals)")
print("Correlation:", corr)
print("p-value:", pval)
if pval < 0.05:
    print(">> Sonuç: H0 reddedilir, log(GDP) ile madalya sayısı arasında anlamlı ilişki var.")
else:
    print(">> Sonuç: H0 reddedilemedi, anlamlı ilişki yok.")


# ------------------------------------------------------------
# AŞAMA 11: Hipotez Testi 2
# Yüksek GDP'li ülkeler (Top 30%) ile düşük GDP'li ülkelerin
# madalya sayıları farklı mı?
# ------------------------------------------------------------

# Ülke-yıl düzeyinde değil de biraz daha düzgün karşılaştırma için
# istersen aynı yıl içindeki ülkeleri kıyaslayabilirsin, ama basit versiyon:
subset2 = merged.dropna(subset=["GDP", "Total_Medals"]).copy()
threshold = subset2["GDP"].quantile(0.70)

high = subset2[subset2["GDP"] >= threshold]["Total_Medals"]
low = subset2[subset2["GDP"] < threshold]["Total_Medals"]

t_stat, pval = ttest_ind(high, low, equal_var=False)

print("\nHipotez 2 - High GDP vs Low GDP countries (medals)")
print("High GDP mean medals:", high.mean())
print("Low GDP mean medals:", low.mean())
print("t-stat:", t_stat)
print("p-value:", pval)
if pval < 0.05:
    print(">> Sonuç: H0 reddedilir, yüksek GDP'li ülkeler istatistiksel olarak daha fazla madalya alıyor.")
else:
    print(">> Sonuç: H0 reddedilemedi, gruplar arasında anlamlı fark yok.")


# ------------------------------------------------------------
# AŞAMA 12: Regresyon Modeli
# Total_Medals ~ GDP + GDP_per_capita + Population
# ------------------------------------------------------------

reg_data = merged[["Total_Medals", "GDP", "GDP_per_capita", "Population"]].dropna().copy()

# Log transformlar (scale'leri küçültmek için):
reg_data["log_GDP"] = np.log10(reg_data["GDP"])
reg_data["log_GDP_per_capita"] = np.log10(reg_data["GDP_per_capita"])
reg_data["log_Population"] = np.log10(reg_data["Population"])

X = reg_data[["log_GDP", "log_GDP_per_capita", "log_Population"]]
X = sm.add_constant(X)
y = reg_data["Total_Medals"]

model = sm.OLS(y, X).fit()
print("\nOLS Regression Results:")
print(model.summary())


# ------------------------------------------------------------
# AŞAMA 13: Overperformer / Underperformer Ülkeler
# ------------------------------------------------------------
# Model yardımıyla beklenen medal sayısı ve residual hesaplayalım:
reg_data["Predicted_Medals"] = model.predict(X)
reg_data["Residual"] = reg_data["Total_Medals"] - reg_data["Predicted_Medals"]

# Residual'ı merged ile birleştirelim ki ülke isimlerini görelim:
# reg_data index'i merged ile uyumlu değil, bu yüzden join yapmayacağız,
# bunun yerine NOC/Country_Name'i de taşıyalım:

# Yeni bir merge için, Index paylaşalım:
temp = merged[["NOC", "Country_Name", "Year"]].join(reg_data[["Total_Medals", "Predicted_Medals", "Residual"]])

# Residual en yüksek (overperform) ülkeler:
overperformers = temp.sort_values("Residual", ascending=False).head(15)
underperformers = temp.sort_values("Residual", ascending=True).head(15)

print("\nTop 15 Overperformers (gerçek > beklenti):")
display(overperformers)

print("\nTop 15 Underperformers (gerçek < beklenti):")
display(underperformers)


FileNotFoundError: [Errno 2] No such file or directory: 'athlete_events.csv'